<a href="https://www.kaggle.com/code/rputtam/classification-using-spark-mllib?scriptVersionId=222771026" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-status-prediction/loan_data.csv


In [ ]:
#### test changes sync

## The objective of this notebook is to predict loan approval status of members.
#### **Prediction Algorithm**: Logistic Regression
#### **Library**: spark MLib

The notebook adheres to the following steps.
1. Installing and importing packages
2. Load the data
3. Data Exploration (EDA)
4. Data Splitting
5. Data Preprocessing
6. Model Building
7. Model Summary - coefficients and pValues
8. Model Prediction
9. Model Evaluation
10. Saving model artifact
11. Applying the model on testdata

## Install and import necessary packages

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849767 sha256=e54501124a84a6e938946dcdd66b64236223f9fbae658c62dbed8b8aba521d19
  Stored in directory: /root/.cache/pip/wheels/d9/1c/98/31e395a42d1735d18d42124971ecbbade844b50bb9845b6f4a
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Creating spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('classification with MLlib').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/16 03:05:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Importing packages
from pyspark.sql.types import *
import pyspark.sql.functions as F
import seaborn as sns
import matplotlib.pyplot as plt

# Data loading

In [5]:
df=spark.read.csv('/kaggle/input/loan-status-prediction/loan_data.csv',inferSchema=True,header=True)

In [6]:
df.describe()

DataFrame[summary: string, Loan_ID: string, Gender: string, Married: string, Dependents: string, Education: string, Self_Employed: string, ApplicantIncome: string, CoapplicantIncome: string, LoanAmount: string, Loan_Amount_Term: string, Credit_History: string, Property_Area: string, Loan_Status: string]

 # Data Exploration

1. Dataframe size
2. Get the field names and their data types, and, number of numeric and categorical features
3. A glimpse of data
4. Missing values count
5. Target variable distribution
6. Continuous variables distribution - normality check and boxplot for outliers
7. Categorical variables distribution
8. Predictors relation with target

In [ ]:
print('Records:',df.count(),'\nColumns:', len(df.columns))

In [ ]:
df.printSchema()

In [ ]:
numeric_data_types=['IntegerType','DoubleType','FloatType','LongType']
string_data_types=['StringType']

cat_cols=0
num_cols=0
other_cols=0

num_cols_list=[]
cat_cols_list=[]
oth_cols_list=[]

for field in df.schema.fields:
    if str(field.dataType)[:-2] in string_data_types:
        cat_cols_list.append(field.name)
        cat_cols+=1
    elif str(field.dataType)[:-2] in numeric_data_types:
        num_cols_list.append(field.name)
        num_cols+=1
    else:
        oth_cols_list.append(field.name)
        other_cols+1
print('Numeric cols:', num_cols,num_cols_list,'\nCategorical Cols:', cat_cols, cat_cols_list,'\nOther Cols:',other_cols,oth_cols_list)  

In [ ]:
#df.show(5)

## Data Glimpse

In [ ]:
df.toPandas().head()
# numeric variables such as income, loan amount, term are considered as strings, but should be converted to numeric data types which is done in data preprocessing step

## Missing values Summary

In [ ]:
missing_values_dict={}
for c in df.columns:
    missing_values_dict[c]=df.agg(F.count(F.when(F.isnan(c) | F.col(c).isNull(),c))).collect()[0][0]
    
missing_values_dict

## Target class distribution

In [ ]:
# Creating target variable
df = df.withColumn('target', F.when(F.col('Loan_Status') == 'Y',1).otherwise(0))
print(df.select('target').dtypes)
df.agg(F.mean('target')).collect()[0][0]

In [ ]:
total_count=df.count()
# Get the target class distribution
df_prop = df.groupBy(['Loan_Status'])\
.agg(F.count('Loan_ID').alias('appl_ct'))\
.withColumn('prop',F.round(F.col('appl_ct')/total_count*100,2))\

df_prop.show()

In [ ]:
df_target_stats=df_prop.toPandas()
df_target_stats.head()

In [ ]:
sns.catplot(data=df_target_stats,x='Loan_Status',y='appl_ct',kind='bar')

plt.xlabel('Loan Status')
plt.ylabel('Count')
plt.title('Distribution of Loan Status')

## Summary Statistics

In [ ]:
df.describe().toPandas().head()

In [ ]:
df.toPandas().describe()

In [ ]:
df.toPandas().describe(include='object')

In [ ]:
string_fields = [col for col,dtype in df.dtypes if dtype =='string']
string_fields.remove('Loan_ID')
for string_field in string_fields:
    df.select(string_field).distinct().show(5)

In [ ]:
df_p=df.toPandas()

In [ ]:
# Get the levels of caterogical variables
for i in string_fields:
    print(i,(df_p[i].nunique(),df_p[i].unique()))
#df_p.Gender.value_counts()

## Data distribution plots of continuous variables

In [ ]:
num_cols_list

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

fig,axs = plt.subplots(1,5,figsize=(15,5))
for i,col in enumerate(num_cols_list):
    sns.kdeplot(data=df_p[col],ax=axs[i],fill=True)
plt.tight_layout()
plt.show()

In [ ]:
#Nuemric variables box plot to cehck for outliers
df_p.boxplot(column=num_cols_list)

In [ ]:
df_p.boxplot(column=['LoanAmount','Loan_Amount_Term'])
# It can be seen from the below plot that the range of these numeric variables is wide and there seem to be outliers for loan_amount_term

In [ ]:
df_p.boxplot(column=['ApplicantIncome','CoapplicantIncome'])
# There seem to be instances where coapplicant income is higher than applicant income

In [ ]:
cat_cols_list

## Categorical variables distribution plot

In [ ]:
# Catgeorical variables counts plot
fig,axes=plt.subplots(nrows=1,ncols=len(cat_cols_list)-1,figsize=(15,5))
for i,col in enumerate(cat_cols_list[1:]):
    cat_counts =df_p[col].value_counts()
    cat_counts.plot(kind='bar',ax=axes[i])

plt.tight_layout()
plt.show()

## Predictors relationship with target

In [ ]:
#Distributions 
fig,axes = plt.subplots(nrows=1,ncols=len(num_cols_list),figsize=(15,5))
for i,col in enumerate(num_cols_list):
    sns.histplot(data=df_p,ax=axes[i],x=col,kde=True,hue='Loan_Status')
plt.tight_layout()
plt.show()

In [ ]:
fig,axes =plt.subplots(nrows=1,ncols=len(num_cols_list),figsize=(15,5))
for i,col in enumerate(num_cols_list):
    sns.stripplot(df_p,x='Loan_Status',y=col,ax=axes[i])
plt.tight_layout()
plt.show()

In [ ]:
#df_p.Loan_Status=df_p.Loan_Status.map(dict(Y=1,N=0))

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=len(cat_cols_list)-1,figsize=(15,5))
for i,col in enumerate(cat_cols_list):
    if col not in ['Loan_Status','Loan_ID']:
        sns.countplot(df_p,ax=axes[i],x='Loan_Status',hue=col)
plt.tight_layout()
plt.show()

# Data Preprocessing

The objective is to have a pipeline with the preprocessing and model building steps that can be used on any similar dataset# Steps

1. Convert all numeric to double and categorical to string
2. Impute missing values - numerical with median and categorical with 'unknown'
3. String indexer to make give numeric labels to categorical variables
4. One hot encoding before applying logistic regression model

In [8]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: double (nullable = true)
 |-- Loan_Amount_Term: double (nullable = true)
 |-- Credit_History: double (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [7]:
from pyspark.ml import Estimator, Transformer, Pipeline
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml.param.shared import HasInputCols, HasOutputCols
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql.functions import col, median, when,lit
from pyspark.sql.types import DoubleType, StringType
from pyspark.ml.feature import Imputer

In [9]:
import json

#### Custom Estimator and Transformer to change datatypes of specified columns into numeric and string datatypes


* Estimator: This is the main class that DataTypeConverter inherits from, providing the basic structure and functionality of an estimator in the PySpark ML pipeline.
* DefaultParamsReadable: This mixin class provides functionality for reading parameters from a saved model. It allows the DataTypeConverter to be loaded from disk after being saved as part of a pipeline.
* DefaultParamsWritable: This mixin class provides functionality for writing parameters when saving a model. It allows the DataTypeConverter to be saved to disk as part of a pipeline.
* Param: Defines parameters for ML components, specifying name, description, and type; ensures type safety and provides documentation. _setDefault() & Getters/Setters: Set default parameter values and provide controlled access, ensuring seamless integration with PySpark's ML Pipelines.


In [10]:
class DataTypeConverter(Estimator, DefaultParamsReadable, DefaultParamsWritable):
    
    numericCols = Param(Params._dummy(), "numericCols", "Columns to be converted to DoubleType",
                        typeConverter=TypeConverters.toListString)
    categoricalCols = Param(Params._dummy(), "categoricalCols", "Columns to be converted to StringType",
                            typeConverter=TypeConverters.toListString)

    def __init__(self, numericCols=None, categoricalCols=None):
        super(DataTypeConverter, self).__init__()
        self._setDefault(numericCols=[], categoricalCols=[])
        self.setNumericCols(numericCols)
        self.setCategoricalCols(categoricalCols)

    def setNumericCols(self, value):
        return self._set(numericCols=value)

    def getNumericCols(self):
        return self.getOrDefault(self.numericCols)

    def setCategoricalCols(self, value):
        return self._set(categoricalCols=value)

    def getCategoricalCols(self):
        return self.getOrDefault(self.categoricalCols)
        
    def _fit(self, dataset):
        return DataTypeConverterModel(self.getNumericCols(), self.getCategoricalCols())

class DataTypeConverterModel(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    numericCols = Param(Params._dummy(), "numericCols", "Columns to be converted to DoubleType",
                        typeConverter=TypeConverters.toListString)
    categoricalCols = Param(Params._dummy(), "categoricalCols", "Columns to be converted to StringType",
                            typeConverter=TypeConverters.toListString)
    
    def __init__(self, numericCols=None, categoricalCols=None):
        super(DataTypeConverterModel, self).__init__()
        self._setDefault(numericCols=[], categoricalCols=[])
        self.setNumericCols(numericCols)
        self.setCategoricalCols(categoricalCols)

    def setNumericCols(self, value):
        return self._set(numericCols=value)

    def getNumericCols(self):
        return self.getOrDefault(self.numericCols)

    def setCategoricalCols(self, value):
        return self._set(categoricalCols=value)

    def getCategoricalCols(self):
        return self.getOrDefault(self.categoricalCols)
        
    def _transform(self,dataset):
        for col_name in self.getNumericCols():
            dataset = dataset.withColumn(col_name,col(col_name).cast(DoubleType()))
        for col_name in self.getCategoricalCols():
            dataset = dataset.withColumn(col_name,col(col_name).cast(StringType()))
        return dataset

In [11]:
class CustomImputer(Estimator, HasInputCols, HasOutputCols,
                    DefaultParamsReadable, DefaultParamsWritable):
    
    inputCols = Param(Params._dummy(), "inputCols", "Columns to be checked and imputed",
                        typeConverter=TypeConverters.toListString)

        
    def __init__(self, inputCols=None):
        super(CustomImputer, self).__init__()
        self._setDefault(inputCols=[])
        self.setInputCols(inputCols)

    def setInputCols(self, value):
        return self._set(inputCols=value)

    def getInputCols(self):
        return self.getOrDefault(self.inputCols)
        
    def _fit(self, dataset):
        "get numeric cols from inputcols list and get the fill value as median aka numericaimputer"
        "get categorical cols from inputcols list and get the fill value aka categoricalimputer"
        numericCols = [field.name for field in dataset.schema.fields if isinstance(field.dataType, DoubleType)]
        categoricalCols = [field.name for field in dataset.schema.fields if isinstance(field.dataType, StringType)]
        # Initialize an empty dictionary to store the medians
        median_values = {}

        # Iterate through the numeric columns and calculate median
        for column in numericCols:
        # Calculate the median
            median_value = df.select(median(col(column))).collect()[0][0]
    
        # Store the median in the dictionary
            median_values[column] = median_value
        return CustomImputerModel(inputCols=self.getInputCols(), 
                                  numericCols=numericCols,
                                  categoricalCols=categoricalCols,
                                  numericFill=median_values,
                                 categoricalFill='unknown')

class CustomImputerModel(Transformer, HasInputCols, HasOutputCols):
    inputCols = Param(Params._dummy(),"inputCols","InputColumns to impute",
                     typeConverter= TypeConverters.toListString)
    numericCols = Param(Params._dummy(), "numericCols", "Numeric columns to impute",
                        typeConverter=TypeConverters.toListString)
    categoricalCols = Param(Params._dummy(), "categoricalCols", "Categorical columns to impute",
                            typeConverter=TypeConverters.toListString)
    numericFill = Param(Params._dummy(), "numericFill", "JSON string of numeric fill values")
    categoricalFill = Param(Params._dummy(), "categoricalFill", "Fill value for categorical columns",
                            typeConverter=TypeConverters.toString)
    
    def __init__(self, inputCols=None, numericCols=None, categoricalCols=None, numericFill=None, categoricalFill=None):
        super(CustomImputerModel, self).__init__()
        self._setDefault(inputCols=[],numericCols=[], categoricalCols=[], numericFill={}, categoricalFill="unknown")
        self.setInputCols(inputCols)
        self.setNumericCols(numericCols)
        self.setCategoricalCols(categoricalCols)
        self.setNumericFill(numericFill)
        self.setCategoricalFill(categoricalFill)

    def setInputCols(self,value):
        return self._set(inputCols=value)

    def setNumericCols(self, value):
        return self._set(numericCols=value)

    def getNumericCols(self):
        return self.getOrDefault(self.numericCols)

    def setCategoricalCols(self, value):
        return self._set(categoricalCols=value)

    def getCategoricalCols(self):
        return self.getOrDefault(self.categoricalCols)

    def setNumericFill(self, value):
        return self._set(numericFill=json.dumps(value))

    def getNumericFill(self):
        return json.loads(self.getOrDefault(self.numericFill))

    def setCategoricalFill(self, value):
        return self._set(categoricalFill=value)

    def getCategoricalFill(self):
        return self.getOrDefault(self.categoricalFill)

   
    def _transform(self, dataset):
        
        numericFill = self.getNumericFill()
        for column in self.getNumericCols():
            
            if column in self.getInputCols():
                dataset = dataset.withColumn(column, when(col(column).isNull(), lit(numericFill[column])).otherwise(col(column)))
        
        # Impute categorical columns
        for column in self.getCategoricalCols():
            if column in self.getInputCols():
                dataset = dataset.withColumn(column, when(col(column).isNull(), lit(self.getCategoricalFill())).otherwise(col(column)))
        
        return dataset

In [ ]:
df.toPandas().head(2)

In [12]:
data_type_converter = DataTypeConverter(numericCols=['ApplicantIncome','CoapplicantIncome',
                                                    'Loanamount','Loan_Amount_Term'], 
                                        categoricalCols=['Gender', 'Married','Dependents','Education','Self_Employed',
                                                       'Property_Area','Loan_Status' ])

In [13]:
custom_imputer = CustomImputer(inputCols=['ApplicantIncome','CoapplicantIncome',
                                        'LoanAmount','Loan_Amount_Term', 
                                        'Gender', 'Married','Dependents','Education',
                                          'Self_Employed','Credit_History',
                                            'Property_Area','Loan_Status' ])

In [14]:
pipeline=Pipeline(stages=[data_type_converter,custom_imputer])

In [15]:
preprocessed_pipeline=pipeline.fit(df)

['ApplicantIncome', 'CoapplicantIncome', 'Loanamount', 'Loan_Amount_Term', 'Credit_History']
['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']


In [16]:
preprocessed_df=preprocessed_pipeline.transform(df)

In [17]:
df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'double'),
 ('Loan_Amount_Term', 'double'),
 ('Credit_History', 'double'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [18]:
preprocessed_df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'double'),
 ('CoapplicantIncome', 'double'),
 ('Loanamount', 'double'),
 ('Loan_Amount_Term', 'double'),
 ('Credit_History', 'double'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [20]:
df.toPandas().isnull().sum()

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [19]:
preprocessed_df.toPandas().isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
Loanamount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [ ]:
numericCols = [field.name for field in df.schema.fields if isinstance(field.dataType, DoubleType)]
categoricalCols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
        # Initialize an empty dictionary to store the medians

In [ ]:
numericCols

In [ ]:
categoricalCols

In [ ]:
class CustomImputer(Estimator, HasInputCols, HasOutputCols,
                    DefaultParamsReadable, DefaultParamsWritable):
    
    inputCols = Param(Params._dummy(), "inputCols", "Columns to be checked and imputed",
                        typeConverter=TypeConverters.toListString)

        
    def __init__(self, inputCols=None):
        super(CustomImputer, self).__init__()
        self._setDefault(inputCols=[])
        self.setInputCols(inputCols)

    def setInputCols(self, value):
        return self._set(inputCols=value)

    def getInputCols(self):
        return self.getOrDefault(self.inputCols)
        
    def _fit(self, dataset):
        "get numeric cols from inputcols list and get the fill value as median aka numericaimputer"
        "get categorical cols from inputcols list and get the fill value aka categoricalimputer"
        numericCols = [field.name for field in dataset.schema.fields if isinstance(field.dataType, DoubleType)]
        categoricalCols = [field.name for field in dataset.schema.fields if isinstance(field.dataType, StringType)]
        print(numericCols)
        print(categoricalCols)
        # Initialize an empty dictionary to store the medians
        median_values = {}

        # Iterate through the numeric columns and calculate median
        for column in numericCols:
        # Calculate the median
            median_value = df.select(median(col(column))).collect()[0][0]
    
        # Store the median in the dictionary
            median_values[column] = median_value
        return CustomImputerModel(inputCols=self.getInputCols(), 
                                  numericCols=numericCols,
                                  categoricalCols=categoricalCols,
                                  numericFill=median_values,
                                 categoricalFill='unknown')

class CustomImputerModel(Transformer, HasInputCols, HasOutputCols):
    inputCols = Param(Params._dummy(),"inputCols","InputColumns to impute",
                     typeConverter= TypeConverters.toListString)
    numericCols = Param(Params._dummy(), "numericCols", "Numeric columns to impute",
                        typeConverter=TypeConverters.toListString)
    categoricalCols = Param(Params._dummy(), "categoricalCols", "Categorical columns to impute",
                            typeConverter=TypeConverters.toListString)
    numericFill = Param(Params._dummy(), "numericFill", "JSON string of numeric fill values")
    categoricalFill = Param(Params._dummy(), "categoricalFill", "Fill value for categorical columns",
                            typeConverter=TypeConverters.toString)
    
    def __init__(self, inputCols=None, numericCols=None, categoricalCols=None, numericFill=None, categoricalFill=None):
        super(CustomImputerModel, self).__init__()
        self._setDefault(inputCols=[],numericCols=[], categoricalCols=[], numericFill={}, categoricalFill="unknown")
        self.setInputCols(inputCols)
        self.setNumericCols(numericCols)
        self.setCategoricalCols(categoricalCols)
        self.setNumericFill(numericFill)
        self.setCategoricalFill(categoricalFill)

    def setInputCols(self,value):
        return self._set(inputCols=value)

    def setNumericCols(self, value):
        return self._set(numericCols=value)

    def getNumericCols(self):
        return self.getOrDefault(self.numericCols)

    def setCategoricalCols(self, value):
        return self._set(categoricalCols=value)

    def getCategoricalCols(self):
        return self.getOrDefault(self.categoricalCols)

    def setNumericFill(self, value):
        return self._set(numericFill=json.dumps(value))

    def getNumericFill(self):
        return json.loads(self.getOrDefault(self.numericFill))

    def setCategoricalFill(self, value):
        return self._set(categoricalFill=value)

    def getCategoricalFill(self):
        return self.getOrDefault(self.categoricalFill)

   
    def _transform(self, dataset):
        
        numericFill = self.getNumericFill()
        print(numericFill)
        print(self.getNumericCols())
        for column in self.getNumericCols():
            
            if column in self.getInputCols():
                print(column)
                dataset = dataset.withColumn(column, when(col(column).isNull(), lit(numericFill[column])).otherwise(col(column)))
        
        # Impute categorical columns
        for column in self.getCategoricalCols():
            if column in self.getInputCols():
                dataset = dataset.withColumn(column, when(col(column).isNull(), lit(self.getCategoricalFill())).otherwise(col(column)))
        
        return dataset

In [ ]:
df.toPandas().isnull().sum()

In [ ]:
custom_imputer = CustomImputer(inputCols=['ApplicantIncome','CoapplicantIncome',
                                        'LoanAmount','Loan_Amount_Term', 
                                        'Gender', 'Married','Dependents','Education',
                                          'Self_Employed','Credit_History',
                                            'Property_Area','Loan_Status' ])

In [ ]:
pipeline=Pipeline(stages=[custom_imputer])

In [ ]:
preprocessed_pipeline=pipeline.fit(df)

In [ ]:
df.toPandas().head(2)

In [ ]:
df.dtypes

In [ ]:
transformed_df=preprocessed_pipeline.transform(df)

##Archive

In [ ]:
transformed_df.toPandas().isnull().sum()

In [ ]:
type(df)

In [ ]:
df.columns

In [ ]:
df.toPandas().head()

In [ ]:
df.toPandas().head()

In [ ]:
df.printSchema()

In [ ]:
#imposing double to all numeric and string to all string fields
print(num_cols_list,'\n',cat_cols_list,'\n',oth_cols_list)

Let's impute missing values

In [ ]:
for col in num_cols_list:
    df = df.withColumn(col, df[col].cast(DoubleType()))

In [ ]:
df.printSchema()

In [ ]:
for col in cat_cols_list:
    df = df.withColumn(col, df[col].cast(StringType()))

In [ ]:
df.printSchema()

In [ ]:
first, load the dataframe
check the fields, their datatypes and missing values
next, convert all numeric to double types
and categorical to string, even though it has numeric values (as in 1,2,3- but it's category)
next, impute missing values
next,do one hot encoding on categorical variables
                                                              

In [ ]:
df.toPandas().head(2)

In [ ]:
from pyspark.ml.feature import Imputer

In [ ]:
Imputer(inputCols=["value"], outputCols=["value_imputed"]).setStrategy("median")

In [ ]:
imputer = Imputer(inputCols=num_cols_list, outputCols=num_cols_list).setStrategy("median")

In [ ]:
df_imputed = imputer.fit(df).transform(df)

In [ ]:
df.toPandas().isnull().sum()

In [ ]:
df_imputed.toPandas().isnull().sum()

In [ ]:
Imputing categorical feaures, the imputer doesnt work, hecne creating custom transformer

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, Transformer, Estimator
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params
from pyspark.sql.functions import col, lit
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable

In [ ]:
from pyspark.ml import Estimator, Model, Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCols, Param, Params
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql.functions import col

class CustomImputerEstimator(Estimator, HasInputCols, HasOutputCols, DefaultParamsReadable, DefaultParamsWritable):
    fillValue = Param(Params._dummy(), "fillValue", "Value to replace missing values with")

    def __init__(self, inputCols=None, outputCols=None, fillValue="missing"):
        super(CustomImputerEstimator, self).__init__()
        self._setDefault(fillValue="missing")
        self.setParams(inputCols=inputCols, outputCols=outputCols, fillValue=fillValue)
    
    def setParams(self, inputCols=None, outputCols=None, fillValue="missing"):
        if inputCols is not None:
            self.setInputCols(inputCols)
        if outputCols is not None:
            self.setOutputCols(outputCols)
        if fillValue is not None:
            self.setFillValue(fillValue)
        return self
    
    def getFillValue(self):
        return self.getOrDefault(self.fillValue)

    def setFillValue(self, value):
        return self._set(fillValue=value)
    
    def setInputCols(self, value):
        return self._set(inputCols=value)
    
    def setOutputCols(self, value):
        return self._set(outputCols=value)
    
    def _fit(self, dataset):
        return CustomImputerModel(inputCols=self.getInputCols(), 
                                  outputCols=self.getOutputCols(), 
                                  fillValue=self.getFillValue())

class CustomImputerModel(Model, HasInputCols, HasOutputCols, DefaultParamsReadable, DefaultParamsWritable):
    fillValue = Param(Params._dummy(), "fillValue", "Value to replace missing values with")

    def __init__(self, inputCols=None, outputCols=None, fillValue="missing"):
        super(CustomImputerModel, self).__init__()
        self._setDefault(fillValue="missing")
        self.setParams(inputCols=inputCols, outputCols=outputCols, fillValue=fillValue)
    
    def setParams(self, inputCols=None, outputCols=None, fillValue="missing"):
        if inputCols is not None:
            self.setInputCols(inputCols)
        if outputCols is not None:
            self.setOutputCols(outputCols)
        if fillValue is not None:
            self.setFillValue(fillValue)
        return self
    
    def getFillValue(self):
        return self.getOrDefault(self.fillValue)

    def setFillValue(self, value):
        return self._set(fillValue=value)
    
    def setInputCols(self, value):
        return self._set(inputCols=value)
    
    def setOutputCols(self, value):
        return self._set(outputCols=value)
    
    def _transform(self, dataset):
        inputCols = self.getInputCols()
        outputCols = self.getOutputCols()
        fillValue = self.getFillValue()
        
        for inputCol, outputCol in zip(inputCols, outputCols):
            dataset = dataset.withColumn(outputCol, col(inputCol).cast("string")).na.fill({outputCol: fillValue})
        return dataset

# Usage in a pipeline
from pyspark.ml import Pipeline


# Create the custom imputer estimator
imputer = CustomImputerEstimator(inputCols=cat_cols_list, outputCols=cat_cols_list, fillValue="Unknown")

# Create a pipeline with the imputer
pipeline = Pipeline(stages=[imputer])

# Fit the pipeline on the training data
model = pipeline.fit(df_train)

# Save the pipeline model
model.save("path/to/save/pipeline_model")

# Later, load the model and transform the test data
from pyspark.ml import PipelineModel
loaded_model = PipelineModel.load("path/to/save/pipeline_model")
imputed_test_df = loaded_model.transform(df_test)

# Show the result
imputed_test_df.show()

In [ ]:
# Later, load the model and transform the test data
from pyspark.ml import PipelineModel
loaded_model = PipelineModel.load("path/to/save/pipeline_model")

# Create a sample test DataFrame
data_test = [(5, None), (6, "b"), (7, None)]
df_test = spark.createDataFrame(data_test, ["id", "category"])

# Transform the test data
imputed_test_df = loaded_model.transform(df_test)

# Show the result
imputed_test_df.show()

In [ ]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCols, Param, Params
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql.functions import col

class CustomImputer(Transformer, HasInputCols, HasOutputCols, DefaultParamsReadable, DefaultParamsWritable):
    fillValue = Param(Params._dummy(), "fillValue", "Value to replace missing values with")

    def __init__(self, inputCols=None, outputCols=None, fillValue="missing"):
        super(CustomImputer, self).__init__()
        self._setDefault(fillValue="missing")
        self.setParams(inputCols=inputCols, outputCols=outputCols, fillValue=fillValue)
    
    def setParams(self, inputCols=None, outputCols=None, fillValue="missing"):
        if inputCols is not None:
            self.setInputCols(inputCols)
        if outputCols is not None:
            self.setOutputCols(outputCols)
        if fillValue is not None:
            self.setFillValue(fillValue)
        return self
    
    def getFillValue(self):
        return self.getOrDefault(self.fillValue)

    def setFillValue(self, value):
        return self._set(fillValue=value)
    
    def setInputCols(self, value):
        return self._set(inputCols=value)
    
    def setOutputCols(self, value):
        return self._set(outputCols=value)
    
    def _transform(self, dataset):
        inputCols = self.getInputCols()
        outputCols = self.getOutputCols()
        fillValue = self.getFillValue()
        
        for inputCol, outputCol in zip(inputCols, outputCols):
            dataset = dataset.withColumn(outputCol, col(inputCol).cast("string")).na.fill({outputCol: fillValue})
        return dataset


# Instantiate the CustomImputer
imputer_cat = CustomImputer(inputCols=cat_cols_list, outputCols=cat_cols_list, fillValue="Unknown")

# Apply the transformer
imputed_cat_df = imputer_cat.transform(df)

# Show the result
#imputed_cat_df.show()

In [ ]:
imputed_cat_df.toPandas().isnull().sum()

In [ ]:
df.toPandas().isnull().sum()

In [ ]:
# Instantiate the CustomImputer
imputer_cat = CustomImputer(inputCol=cat_cols_list, outputCol=cat_cols_list, fillValue="Unknown")

# Apply the transformer
imputed_cat_df = imputer_cat.transform(df)

# Show the result
imputed_cat_df.show()

#### Archive

In [ ]:
# Let's split the data into train, valid and test datatsets
train,valid,test=df.randomSplit(weights=[0.7,0.2,0.1],seed=12345)

In [ ]:
print('Train dataset',train.count(),train.agg(F.sum('target')).collect()[0][0])
print('Valid dataset',valid.count(),valid.agg(F.sum('target')).collect()[0][0])
print('Test dataset',test.count(),test.agg(F.sum('target')).collect()[0][0])

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

In [ ]:
from pyspark.ml.feature import StandardScaler,OneHotEncoder,StringIndexer, VectorAssembler, Imputer,ChiSqSelector